In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# connect TFRecords to graph

In [2]:
data_path = '/shared/TORCS_train.tfrecords'

feature = {'train/fast': tf.FixedLenFeature([], tf.float32),
    'train/dist_RR': tf.FixedLenFeature([], tf.float32),
    'train/dist_MM': tf.FixedLenFeature([], tf.float32),
    'train/dist_LL': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_RR': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_MR': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_ML': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_LL': tf.FixedLenFeature([], tf.float32),
    'train/dist_R': tf.FixedLenFeature([], tf.float32),
    'train/dist_L': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_R': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_M': tf.FixedLenFeature([], tf.float32),
    'train/toMarking_L': tf.FixedLenFeature([], tf.float32),
    'train/angle': tf.FixedLenFeature([], tf.float32),
    'train/image': tf.FixedLenFeature([], tf.string)}

# Create a list of filenames and pass it to a queue
filename_queue = tf.train.string_input_producer([data_path], num_epochs=100)
# Define a reader and read the next record
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
# Decode the record read by the reader
features = tf.parse_single_example(serialized_example, features=feature)
# Convert the image data from string back to the numbers
image = tf.decode_raw(features['train/image'], tf.uint8)

# Cast label data into float32
fast = tf.cast(features['train/fast'], tf.float32)

# rescale all targets to [0,1], numbers were taken from source code of DeepDriving
dist_RR = tf.cast(features['train/dist_RR'], tf.float32) / 95.0 + 0.12
dist_MM = tf.cast(features['train/dist_MM'], tf.float32) / 95.0 + 0.12
dist_LL = tf.cast(features['train/dist_LL'], tf.float32) / 95.0 + 0.12

toMarking_RR = tf.cast(features['train/toMarking_RR'], tf.float32) / 6.8752 - 0.48181
toMarking_MR = tf.cast(features['train/toMarking_MR'], tf.float32) / 6.25 + 0.02
toMarking_ML = tf.cast(features['train/toMarking_ML'], tf.float32) / 6.25 + 0.98
toMarking_LL = tf.cast(features['train/toMarking_LL'], tf.float32) / 6.8752 + 1.48181

dist_R = tf.cast(features['train/dist_R'], tf.float32) / 95.0 + 0.12
dist_L = tf.cast(features['train/dist_L'], tf.float32) / 95.0 + 0.12

toMarking_R = tf.cast(features['train/toMarking_R'], tf.float32) / 5.6249 - 0.34445
toMarking_M = tf.cast(features['train/toMarking_M'], tf.float32) / 6.8752 + 0.39091
toMarking_L = tf.cast(features['train/toMarking_L'], tf.float32) / 5.6249 + 1.34445

angle = tf.cast(features['train/angle'], tf.float32) / 1.1 + 0.5

# Reshape image data into the original shape
image = tf.reshape(image, [210, 280, 3])

# Any preprocessing here ...
image = tf.cast(image, tf.float32)
image = (image - 128.0) / 128.0

# Creates batches by randomly shuffling tensors
#images=1
images, fasts, dist_RRs, dist_MMs, dist_LLs, toMarking_RRs, toMarking_MRs, toMarking_MLs, toMarking_LLs, dist_Rs, dist_Ls, toMarking_Rs, toMarking_Ms, toMarking_Ls, angles = tf.train.shuffle_batch(
                    [image, fast, dist_RR, dist_MM, dist_LL, toMarking_RR, toMarking_MR, toMarking_ML,
                        toMarking_LL, dist_R, dist_L, toMarking_R, toMarking_M, toMarking_L, angle],
                        batch_size=64, capacity=50000, num_threads=8, min_after_dequeue=10000, allow_smaller_final_batch=True)


In [3]:
y_true = tf.concat([tf.expand_dims(fasts,1), tf.expand_dims(dist_RRs,1), tf.expand_dims(dist_MMs,1), tf.expand_dims(dist_LLs,1), 
                    tf.expand_dims(toMarking_RRs,1), tf.expand_dims(toMarking_MRs,1), tf.expand_dims(toMarking_MLs,1),
                    tf.expand_dims(toMarking_LLs,1), tf.expand_dims(dist_Rs,1), tf.expand_dims(dist_Ls,1), tf.expand_dims(toMarking_Rs,1),
                    tf.expand_dims(toMarking_Ms,1), tf.expand_dims(toMarking_Ls,1), tf.expand_dims(angles,1)], axis=1)

# Alexnet

In [8]:
images

<tf.Tensor 'shuffle_batch_1:0' shape=(64, 210, 280, 3) dtype=float32>

In [4]:
from alexnet import alexnet_v2


logits, endpoints = alexnet_v2(images,
               num_classes=1000,
               is_training=True,
               dropout_keep_prob=0.999,
               spatial_squeeze=True,
               scope='alexnet_v2')

y_pred = endpoints['alexnet_v2/fc8']
loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

In [10]:
endpoints

OrderedDict([('alexnet_v2/conv1',
              <tf.Tensor 'alexnet_v2/conv1/Relu:0' shape=(64, 50, 68, 64) dtype=float32>),
             ('alexnet_v2/pool1',
              <tf.Tensor 'alexnet_v2/pool1/MaxPool:0' shape=(64, 24, 33, 64) dtype=float32>),
             ('alexnet_v2/conv2',
              <tf.Tensor 'alexnet_v2/conv2/Relu:0' shape=(64, 24, 33, 192) dtype=float32>),
             ('alexnet_v2/pool2',
              <tf.Tensor 'alexnet_v2/pool2/MaxPool:0' shape=(64, 11, 16, 192) dtype=float32>),
             ('alexnet_v2/conv3',
              <tf.Tensor 'alexnet_v2/conv3/Relu:0' shape=(64, 11, 16, 384) dtype=float32>),
             ('alexnet_v2/conv4',
              <tf.Tensor 'alexnet_v2/conv4/Relu:0' shape=(64, 11, 16, 384) dtype=float32>),
             ('alexnet_v2/conv5',
              <tf.Tensor 'alexnet_v2/conv5/Relu:0' shape=(64, 11, 16, 256) dtype=float32>),
             ('alexnet_v2/pool5',
              <tf.Tensor 'alexnet_v2/pool5/MaxPool:0' shape=(64, 5, 7, 256) dtyp

# Mobilenet

In [4]:
from mobilenet_v1 import mobilenet_v1

logits, endpoints = mobilenet_v1(images, #tf.cast(images, tf.float32),
                 num_classes=1000,
                 dropout_keep_prob=0.999,
                 is_training=True,
                 min_depth=8,
                 depth_multiplier=0.75,
                 conv_defs=None,
                 prediction_fn=tf.contrib.layers.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 scope='MobilenetV1')

y_pred = endpoints['Logits']
loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

# Training

In [ ]:
from time import time

start_t = time()

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

saver = tf.train.Saver(max_to_keep=None)

with tf.Session() as sess:

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)

    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    loss_list = []
    for batch_index in range(140000):
        loss_val, _ = sess.run([loss, train_step])
        loss_list.append((loss_val, time() - start_t))
        if batch_index % 10 == 0:
            np.save('loss.npy', loss_list)
        if (batch_index+1) % 20000 == 0:
            saver.save(sess, "model_alex/{}.ckpt".format(batch_index))
    
    # Stop the threads
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    sess.close()

INFO:tensorflow:model_alex/19999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/39999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/59999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/79999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/99999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/119999.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:model_alex/139999.ckpt is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

with tf.Session() as sess:

    # Initialize all global and local variables
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)

    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for batch_index in range(5):
        img, lbl = sess.run([images, labels])
        print img.max(), img.min()
        break
        img = img.astype(np.uint8)
        for j in range(6):
            plt.subplot(2, 3, j+1)
            plt.imshow(img[j, ...])
            plt.title(str(lbl[j]))
        plt.show()
    # Stop the threads
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
    sess.close()

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


NameError: name 'labels' is not defined

# Mobilenet

In [3]:
! apt-get update; apt-get install wget

Hit:1 http://archive.ubuntu.com/ubuntu xenial InRelease
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]       
Get:3 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]     
Get:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]     
Fetched 306 kB in 0s (448 kB/s)                                                
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.17.1-1ubuntu1.2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [4]:
rm /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/slim/python/slim/nets/mobilenet_v1.py*

In [5]:
! wget https://raw.githubusercontent.com/tensorflow/models/master/slim/nets/mobilenet_v1.py

--2017-07-27 14:18:09--  https://raw.githubusercontent.com/tensorflow/models/master/slim/nets/mobilenet_v1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19467 (19K) [text/plain]
Saving to: 'mobilenet_v1.py.1'

mobilenet_v1.py.1   100%[===================>]  19.01K  --.-KB/s    in 0.03s   

2017-07-27 14:18:09 (668 KB/s) - 'mobilenet_v1.py.1' saved [19467/19467]



In [4]:
from mobilenet_v1 import mobilenet_v1

In [5]:
images

<tf.Tensor 'shuffle_batch:0' shape=(10, 210, 280, 3) dtype=uint8>

In [6]:
logits, endpoints = mobilenet_v1(tf.cast(images, tf.float32),
                 num_classes=1000,
                 dropout_keep_prob=0.999,
                 is_training=True,
                 min_depth=8,
                 depth_multiplier=1.0,
                 conv_defs=None,
                 prediction_fn=tf.contrib.layers.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 scope='MobilenetV1')

In [7]:
y_pred = endpoints['Logits']
y_true = tf.placeholder(shape=[None, 14], dtype=tf.float32)
loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))

{'AvgPool_1a': <tf.Tensor 'MobilenetV1/Logits/AvgPool_1a/AvgPool:0' shape=(10, 1, 1, 1024) dtype=float32>,
 'Conv2d_0': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_0/Relu:0' shape=(10, 105, 140, 32) dtype=float32>,
 'Conv2d_10_depthwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_10_depthwise/Relu:0' shape=(10, 14, 18, 512) dtype=float32>,
 'Conv2d_10_pointwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_10_pointwise/Relu:0' shape=(10, 14, 18, 512) dtype=float32>,
 'Conv2d_11_depthwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_11_depthwise/Relu:0' shape=(10, 14, 18, 512) dtype=float32>,
 'Conv2d_11_pointwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_11_pointwise/Relu:0' shape=(10, 14, 18, 512) dtype=float32>,
 'Conv2d_12_depthwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_12_depthwise/Relu:0' shape=(10, 7, 9, 512) dtype=float32>,
 'Conv2d_12_pointwise': <tf.Tensor 'MobilenetV1/MobilenetV1/Conv2d_12_pointwise/Relu:0' shape=(10, 7, 9, 1024) dtype=float32>,
 'Conv2d_13_depthwise

In [8]:
import tensorflow as tf

sess=tf.Session()
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('mobilenet/mobilenet_v1_0.50_128.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))


# Access saved Variables directly
# print(sess.run('bias:0'))
# This will print 2, which is the value of bias that we saved


# Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
#w1 = graph.get_tensor_by_name("w1:0")
#w2 = graph.get_tensor_by_name("w2:0")
#feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
#op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

#print sess.run(op_to_restore,feed_dict)
#This will print 60 which is calculated 

ValueError: No op named SSTableReaderV2 in defined operations.